In [1]:
using DataFrames
using LinearAlgebra
using Distributions
using CSV
using NPZ
using Base.Threads
using BenchmarkTools
using ProgressMeter
using Plots
using Random
using Pkg

In [2]:
Pkg.develop(path="c:\\Users\\guibi\\.julia\\dev\\MyARCHModels")
using MyARCHModels

   Resolving package versions...
  No Changes to `C:\Users\guibi\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\guibi\.julia\environments\v1.10\Manifest.toml`
Precompiling MyARCHModels
  ✓ MyARCHModels
  1 dependency successfully precompiled in 24 seconds. 108 already precompiled.


In [3]:
data = CSV.read("returns_data.csv", DataFrame)[2:end,2:end]
data_mat = coalesce.(Matrix(data), 0.0)

data_small = data[:,1:50]
data_small_mat = coalesce.(Matrix(data_small), 0.0);

In [4]:
function markowitz_min_var_port(Σ::Matrix{Float64})
    (n, _) = size(Σ)
    SigmaI = inv(Σ)
    e = ones(n)
    den = e' * SigmaI * e
    return (SigmaI * e) / den
end

function markowitz_min_var(Σ::Matrix{Float64})
    w = markowitz_min_var_port(Σ)
    return w' * Σ * w
end

function get_predictions(data::Matrix{Float64})
    (n,d) = size(data)
    pred_cov = [Matrix{Float64}(I(d)) for _ in 1:n]
    @showprogress for time in 12:n
            sliced_data = deepcopy(data[1:time,:])
            spec = DCC{1,1}
            res = fit(spec, sliced_data)
            pred_cov[time] = predict(res)
    end
    return pred_cov
end

function loss(Sigma::Matrix{Float64}, Sigma_hat::Matrix{Float64})
    (n, _) = size(Sigma)
    Sigma_hat_I = inv(Sigma_hat)
    a = tr(Sigma_hat_I * Sigma * Sigma_hat_I) / n
    b = (tr(Sigma_hat_I) /n)^2
    c = 1 / ( tr(inv(Sigma)) / n )
    return a / b - c
end

loss (generic function with 1 method)

In [5]:
function plot_first_5_series_in_subplots(data::AbstractMatrix)
    # Ensure data has at least 5 columns
    num_series = min(size(data, 2), 5)
    
    # Create a layout for 5 rows and 1 column (adjust if fewer than 5 series)
    layout = grid(num_series, 1)  # Creates a layout with num_series rows and 1 column
    
    # Generate the plot
    p = plot(layout=layout, size=(800, 1200))
    
    for i in 1:num_series
        plot!(1:size(data, 1), data[:, i], label="Series $i", lw=2, subplot=i)
    end
    
    display(p)
end

plot_first_5_series_in_subplots (generic function with 1 method)

In [6]:
mod = fit(DCC{1,1}, copy(coalesce.(Matrix(data), 0.0)))
p = 500
R = mod.spec.R;
real_covs = covariances(mod);

In [7]:
univariate_spec = UnivariateARCHModel
simulation_spec = DCC{1, 1}(R, [.96, .02], [GARCH{1, 1}([10e-5, .9, .05]) for _ in 1:p])

DCC{1, 1, GARCH{1, 1}} specification.

───────────────────────
               β₁    α₁
───────────────────────
Parameters:  0.96  0.02
───────────────────────


In [9]:
linear_loss_sim = [0.0 for i in 1:100]

@showprogress for i in 1:100
    sim_data = simulate(simulation_spec, 1000; warmup = 100).data
    est = fit(DCC{1,1}, sim_data, method = :linear)
    est_covs = covariances(est)
    linear_loss_sim[i] = sum([loss(real, est) for (real,est) in zip(real_covs, est_covs)])
end

npzwrite("linear_loss_sim.npy", linear_loss_sim)

Progress: 100%|█████████████████████████████████████████| Time: 1:31:43


In [10]:
analytical_loss_sim = [0.0 for i in 1:100]

@showprogress for i in 1:100
    sim_data = simulate(simulation_spec, 1000; warmup = 100).data
    est = fit(DCC{1,1}, sim_data, method = :largescale)
    est_covs = covariances(est)
    analytical_loss_sim[i] = sum([loss(real, est) for (real,est) in zip(real_covs, est_covs)])
end

npzwrite("anal_loss_sim.npy", analytical_loss_sim)

Progress: 100%|█████████████████████████████████████████| Time: 1:25:18


In [8]:
sample_cov_sim = [0.0 for i in 1:100]

@showprogress for i in 1:100
    sim_data = simulate(simulation_spec, 1000; warmup = 100).data
    est = fit(DCC{1,1}, sim_data, method = :twostep)
    est_covs = covariances(est)
    sample_cov_sim[i] = sum([loss(real, est) for (real,est) in zip(real_covs, est_covs)])
end

npzwrite("cov_loss_sim.npy", sample_cov_sim)

Progress: 100%|█████████████████████████████████████████| Time: 1:28:13


In [9]:
sample_cov_sim

100-element Vector{Float64}:
 1.1708601768826654
 1.1963586565163211
 1.1688816137065576
 1.1846898176522704
 1.1833029623335218
 1.1673979996535566
 1.168789885522228
 1.1637807202852959
 1.1809467546010677
 1.1929418530396965
 1.168367639467903
 1.1990337292680886
 1.1605954284718765
 ⋮
 1.1592932057010497
 1.1721186210213346
 1.1813859046967292
 1.1584371223540997
 1.195394815487476
 1.1973216555930615
 1.1797886822391912
 1.1725401117757583
 1.1948504699441118
 1.1906494685953655
 1.1733779469310943
 1.1840395579933563